In [1]:
import numpy as np 
import pandas as pd

import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.neighbors import NearestNeighbors


In [2]:
movies_df = pd.read_csv('/home/dans/Downloads/pyp/archive/movies_md.csv', low_memory=False)
keywords = pd.read_csv('/home/dans/Downloads/pyp/archive/keywords.csv', low_memory=False)
links = pd.read_csv("/home/dans/Downloads/pyp/archive/links.csv", low_memory=False)
credits = pd.read_csv("/home/dans/Downloads/pyp/archive/credits.csv")


In [3]:

movies_data = movies_df[['id','genres','adult','runtime', 'tagline', 'vote_average','title']]

keywords_data =  keywords['keywords']

credits_data = credits[['cast', 'crew']]





In [4]:
listData = pd.concat([movies_data, keywords_data, credits_data], axis=1)

listData

,id,genres,adult,runtime,tagline,vote_average,title,keywords,cast,crew
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",False,81.0,NaN,7.7,Toy Story,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,...","[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",False,104.0,Roll the dice and unleash the excitement!,6.9,Jumanji,"[{'id': 10090, 'name': 'board game'}, {'id': 1...","[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,15602,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",False,101.0,Still Yelling. Still Fighting. Still Ready for...,6.5,Grumpier Old Men,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392...","[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de..."
3,31357,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",False,127.0,Friends are the people who let you be yourself...,6.1,Waiting to Exhale,"[{'id': 818, 'name': 'based on novel'}, {'id':...","[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de..."
4,11862,"[{'id': 35, 'name': 'Comedy'}]",False,106.0,Just When His World Is Back To Normal... He's ...,5.7,Father of the Bride Part II,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...","[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de..."
...,...,...,...,...,...,...,...,...,...,...
46414,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 10703, 'name': 'tragic love'}]",NaN,NaN
46415,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,...",NaN,NaN
46416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN
46417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN


In [5]:
#movies_df['id'] = movies_df['id'].astype('string')
#movies_df['vote_average'] = movies_df['vote_average'].astype('string')
#movies_df['runtime'] = movies_df['runtime'].astype('string')

#keywords['id'] = keywords['id'].astype('string')

#moviesData = movies_df.merge(keywords, on='id')

In [6]:
columns = ['id','genres','adult','runtime', 'tagline', 'vote_average','keywords','title','cast','crew']

for column in columns:
    listData[column] = listData[column].fillna('')

In [7]:
listData['runtime'] = listData['runtime'].astype('string')
listData['vote_average'] = listData['vote_average'].astype('string')


bases = listData['genres']+""+listData['keywords']+""+listData['tagline']+""+listData['title']+''+listData['runtime']+''+listData['vote_average']

bases




0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4        [{'id': 35, 'name': 'Comedy'}][{'id': 1009, 'n...
                               ...                        
46414               [{'id': 10703, 'name': 'tragic love'}]
46415    [{'id': 2679, 'name': 'artist'}, {'id': 14531,...
46416                                                   []
46417                                                   []
46418                                                   []
Length: 46419, dtype: string

In [8]:
vectorizer = TfidfVectorizer()

vectorizedBases = vectorizer.fit_transform(bases)

from scipy.sparse import csr_matrix

matBases=csr_matrix(vectorizedBases)

matBases



<46419x87382 sparse matrix of type '<class 'numpy.float64'>'
	with 972644 stored elements in Compressed Sparse Row format>

In [9]:
#similarity = cosine_similarity(matBases)

nn = NearestNeighbors(metric='cosine', algorithm='brute')

nn.fit(vectorizedBases)

distances, indices = nn.kneighbors(vectorizedBases, n_neighbors=30)

distances.shape

(46419, 30)

In [10]:
userChoice = input('Enter Choice:')

userChoice


'avatar'

In [11]:
moviesList = listData['title']



In [12]:
getCloseMatch = difflib.get_close_matches(userChoice, moviesList)

matchedMovie = getCloseMatch[0]

matchedMovie

'Avatar'

In [13]:

# Get matched Movie Id

matchedMovieIndex = listData[listData['title'] == matchedMovie]['id'].index[0]

matchedMovieIndex


14551

In [14]:
similarity_score = list(enumerate(distances[matchedMovieIndex]))

sortedMovies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

distances[40000]


array([0.        , 0.43574862, 0.55572185, 0.59381835, 0.65204004,
       0.65882997, 0.66369007, 0.67191868, 0.67238774, 0.67570373,
       0.694564  , 0.71963849, 0.72484026, 0.73872975, 0.74682615,
       0.83473073, 0.86375092, 0.86644672, 0.8677731 , 0.86932667,
       0.87169231, 0.87247886, 0.87265575, 0.87397221, 0.87426641,
       0.87446144, 0.87537744, 0.87592133, 0.87604796, 0.8763575 ])

In [15]:
i = 1
for movie in similarity_score:
    index = movie[0]
    title = movies_df[movies_df.index == index]['title'].values[0]

    print(title)

Toy Story
Jumanji
Grumpier Old Men
Waiting to Exhale
Father of the Bride Part II
Heat
Sabrina
Tom and Huck
Sudden Death
GoldenEye
The American President
Dracula: Dead and Loving It
Balto
Nixon
Cutthroat Island
Casino
Sense and Sensibility
Four Rooms
Ace Ventura: When Nature Calls
Money Train
Get Shorty
Copycat
Assassins
Powder
Leaving Las Vegas
Othello
Now and Then
Persuasion
The City of Lost Children
Shanghai Triad


In [16]:
def find_similar_movies(movie_title, n =29):
    movie_title = difflib.get_close_matches(movie_title, moviesList)[0]

    idx = movies_data[movies_data['title'] == movie_title].index[0]
    distances, indices = nn.kneighbors(vectorizedBases[idx], n_neighbors=n+1)
    return movies_data.iloc[indices[0][1:]][['title','vote_average']]


In [17]:
similar = find_similar_movies('moana')

print(similar)

                                         title  vote_average
11156                            The Hurricane           5.9
4665                              Father Goose           6.5
32588                        The Great Garrick           5.0
8321                            Sadie Thompson           6.4
21548                Bloodlust: Subspecies III           4.5
33753                               The Master           0.0
5462                                      Tabu           7.4
9613                   Pippi in the South Seas           7.0
10237                               Dust Devil           6.7
9259                    Werckmeister Harmonies           7.4
24117                            Tattoo Nation           6.8
28762                   The Coconut Revolution           0.0
26604                          Pagan Love Song           6.0
25641                               On the Ice           7.0
38996                   Coffin Full of Dollars           5.3
12428                   

In [18]:
movies_df

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [19]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/recommend', methods=['GET'])
def recommend():
    movie_title = request.args.get('title')
    n = int(request.args.get('n', 29))
    similar_movies = find_similar_movies(movie_title, n)
    return jsonify(similar_movies.to_dict(orient ='records'))

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.9:5999
Press CTRL+C to quit
 * Restarting with stat
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_port = self._bind_socket(self.shell_sock

SystemExit: 1

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
